In [1]:
import pandas_gbq

In [11]:
from main_agrupamento_nomes import main_agrupamento_nomes
main_agrupamento_nomes('2024-08-31')

2024-08-31
Tratando dados de  2024-08-31
[2024-09-26 20:31:01] Getting dataset from BQ...
Downloading: 100%|██████████|


Processando grupos: 100%|██████████| 5571/5571 [00:07<00:00, 787.79grupo/s] 


In [2]:
import sys
import os

# Verify it has been added
print(sys.path)

['/home/jupyter/Projects/TAM_Ruan', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/home/jupyter/.local/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/home/jupyter/.ipython']


In [3]:

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

import time

from projectutils import read_data 
import sys



In [4]:
#import sys, importlib
#importlib.reload(sys.modules['group_tam_id'])
from group_tam_id import (
    assign_group_ids,
    open_ton, load_to_gbq, create_doc_final, open_init_data,
    execute_with_context,
    init_group_id, 
    deal_merged_places, 
    deal_merged_docs,
    deal_unmerged_places, 
    choose_prefered_document, 
    create_agrupamento_inspecao, 
    grouped_subs_asterisk, 
    final_ajustes)

In [5]:

def create_main_data_abr():

    df = read_data('final_nomes')
    df = df[df['reference_month']==np.datetime64('2024-04-30T00:00:00.000000000')]

    LEVEL_GROUP = ['cod_muni', 'reference_month']
    OUTROS = ['Outros', 'Outros_Pags', 'Outros_SumUp', 'Outros_Stone']

    print('Apenas mmhid de Outro')
    df['merchant_market_hierarchy_id'] = df['merchant_market_hierarchy_id'].where(df['subs_asterisk'].isin(OUTROS), None)


    df = df.drop_duplicates(['subs_asterisk', 'nome_master', 'merchant_market_hierarchy_id'] + LEVEL_GROUP)

    ton = open_ton(read_data, LEVEL_GROUP)
    ton = ton[ton['reference_month']==np.datetime64('2024-04-30T00:00:00.000000000')]

    df = pd.concat([
            df,
            ton,
        ])
    df['nome_master'] = df['nome_master'].str.replace(r'[^A-Z]', '', regex=True)
    df['inicio'] = df['nome_master'].str[:6]

    df['cod_muni'] = df['cod_muni'].fillna(9999)
        #df_orig = df_orig[~df_orig['cod_muni'].isin(saved_munis)]
    df['nome_muni'] = df['nome_muni'].fillna('Desconhecido')
    df['uf'] = df['uf'].fillna('DE')

    df.to_parquet('data/main_data_2024-04-30.parquet')

    
import pandas as pd
import numpy as np
from datetime import datetime

def create_main_data(input_date: str):

    # Convert input date string to datetime object
    reference_date = np.datetime64(datetime.strptime(input_date, '%Y-%m-%d'))

    df = read_data('final_nomes')
    df = df[df['reference_month'] == reference_date]

    LEVEL_GROUP = ['cod_muni', 'reference_month']
    OUTROS = ['Outros', 'Outros_Pags', 'Outros_SumUp', 'Outros_Stone']

    print('Apenas mmhid de Outro')
    df['merchant_market_hierarchy_id'] = df['merchant_market_hierarchy_id'].where(df['subs_asterisk'].isin(OUTROS), None)

    df = df.drop_duplicates(['subs_asterisk', 'nome_master', 'merchant_market_hierarchy_id'] + LEVEL_GROUP)

    ton = open_ton(read_data, LEVEL_GROUP)
    ton = ton[ton['reference_month'] == reference_date]

    df = pd.concat([df, ton])
    df['nome_master'] = df['nome_master'].str.replace(r'[^A-Z]', '', regex=True)
    df['inicio'] = df['nome_master'].str[:6]

    df['cod_muni'] = df['cod_muni'].fillna(9999)
    df['nome_muni'] = df['nome_muni'].fillna('Desconhecido')
    df['uf'] = df['uf'].fillna('DE')

    # Save to parquet file with the input date in the filename
    df.to_parquet(f'data/main_data_{input_date}.parquet')


In [8]:
def main_read_data():
    df_orig = read_data('final_nomes')
    LEVEL_GROUP = ['cod_muni', 'reference_month']
    ton_orig = open_ton(read_data, LEVEL_GROUP)
    df_orig = open_init_data(df_orig, ton_orig, LEVEL_GROUP)
    del ton_orig
    
    #saved_munis = find_saved_munis()
    df_orig['cod_muni'] = df_orig['cod_muni'].fillna(9999)
    #df_orig = df_orig[~df_orig['cod_muni'].isin(saved_munis)]
    df_orig['nome_muni'] = df_orig['nome_muni'].fillna('Desconhecido')
    df_orig['uf'] = df_orig['uf'].fillna('DE')
    return df_orig

def save_main_data():
    df = main_read_data()
    df.to_parquet('data/main_data.parquet')

In [9]:

def group_and_clean(df, LEVEL_GROUP):
    df = df.copy()
    from test_grouping import group_names_df

    print('Tamanho do DataFrame:', df.shape)

    #df['qtd_por_inicio'] = df.groupby(['nome_muni', 'uf'])['nome_master'].transform('count')
    #df = df[df['qtd_por_inicio']>1]

    df = df.sort_values(by=LEVEL_GROUP + ['nome_master']).reset_index(drop=True)

    df = group_names_df(df, group_cols=[
                                        'inicio',
                                        ] + LEVEL_GROUP, 
                        nome_col='nome_master',
                        delete_aux_cols=False,
                        verbose=False)
    
    if LEVEL_GROUP:
        df['agrupamento_nome_1'] = df['res_aux'].apply(lambda x: ', '.join(x)) + df[LEVEL_GROUP].astype(str).apply(lambda x: '|'.join(x), axis=1)
    else:
        df['agrupamento_nome_1'] = df['res_aux'].apply(lambda x: ', '.join(x))
    
    df = df.drop(columns=['resultado_prefixes', 'res_aux', 'retirado_de_resultado'])
    
    return df

def save_name_grouping(df, cod_muni, output_dir):
    print('-----------\n Analyzing:', cod_muni)
    df = df.copy()
    LEVEL_GROUP = []
    df = group_and_clean(df, LEVEL_GROUP)
    df.to_parquet(f"{output_dir}/part_{cod_muni}.parquet")
    print('Saved:', cod_muni)

def vm_save_grouping(read_data_func):
    output_dir = r'data/nomes_agrupados/2024-04-30'
    df = read_data(f'main_data_2024-04-30')
    LEVEL_GROUP = ['cod_muni', 'reference_month']

    df = df[['nome_master', 'inicio', 'cod_muni', 'reference_month']].drop_duplicates()
    unique_cod_muni = df['cod_muni'].unique()
    for muni in unique_cod_muni:
        save_name_grouping(df[df['cod_muni']==muni], muni, output_dir)
        


#vm_save_grouping(read_data)

In [10]:
def vm_save_grouping(read_data_func, input_date):
    output_dir = f'data/nomes_agrupados/{input_date}'
    df = read_data(f'main_data_{input_date}')
    LEVEL_GROUP = ['cod_muni', 'reference_month']

    df = df[['nome_master', 'inicio', 'cod_muni', 'reference_month']].drop_duplicates()
    unique_cod_muni = df['cod_muni'].unique()
    for muni in unique_cod_muni:
        save_name_grouping(df[df['cod_muni']==muni], muni, output_dir)
        

In [11]:
#vm_save_grouping(read_data, '2024-03-31')

In [31]:
def main(date):
    create_main_data(date)
    vm_save_grouping(read_data, date)
    
create_main_data('2024-08-31')    

Apenas mmhid de Outro


In [33]:
#vm_save_grouping(read_data, '2024-08-31')

In [12]:
#vm_save_grouping(read_data, '2024-04-30')

In [13]:

def vm_load_grouping():
    import os
    start_time = time.time()
    output_dir = r'data/nomes_agrupados'
    df = pd.concat([pd.read_parquet(f"{output_dir}/{f}") for f in os.listdir(output_dir)])
    print(f'Loading data in {time.time()  - start_time:.2f} seconds.')
    df['agrupamento_nome_1'] = df['agrupamento_nome_1'] + df['cod_muni'].astype(str)
    return df


def vm_load_grouping_part(input_date, cod_munis):
    import os
    start_time = time.time()
    output_dir = fr'data/nomes_agrupados/{input_date}' 
    df = pd.concat([pd.read_parquet(f"{output_dir}/part_{cod_muni}.parquet") for cod_muni in cod_munis])
    print(f'Loading data in {time.time()  - start_time:.2f} seconds.')
    df['agrupamento_nome_1'] = df['agrupamento_nome_1'] + df['cod_muni'].astype(str)
    return df


In [14]:
def add_grouped(df, LEVEL_GROUP):
    df = df.copy()
    print('Tamanho do DataFrame:', df.shape)
    df['cod_muni'] = df['cod_muni'].fillna(9999)
    
    df = df.merge(vm_load_grouping(), on=['inicio', 'cod_muni', 'nome_master'], how='inner', indicator=True)
    display(df[df['_merge']!='both'])
    return df

def add_grouped_part(df, input_date, cod_munis):
    df = df.copy()
    print('Tamanho do DataFrame:', df.shape)
    df = df.merge(vm_load_grouping_part(input_date, cod_munis), on=['inicio', 'cod_muni', 'nome_master'], how='inner', indicator=True)
    return df

In [15]:

OUTROS = ['Outros', 'Outros_Pags', 'Outros_SumUp', 'Outros_Stone']


In [16]:
def main_read_data():
    df_orig = read_data('final_nomes')
    LEVEL_GROUP = ['cod_muni']
    ton_orig = open_ton(read_data, LEVEL_GROUP)
    df_orig = open_init_data(df_orig, ton_orig, LEVEL_GROUP)
    del ton_orig
    
    #saved_munis = find_saved_munis()
    df_orig['cod_muni'] = df_orig['cod_muni'].fillna(9999)
    #df_orig = df_orig[~df_orig['cod_muni'].isin(saved_munis)]
    df_orig['nome_muni'] = df_orig['nome_muni'].fillna('Desconhecido')
    df_orig['uf'] = df_orig['uf'].fillna('DE')
    return df_orig

def save_main_data():
    df = main_read_data()
    df.to_parquet('data/main_data.parquet')
    

In [17]:

def create_doc_final(df):

    df['qtd_mmhid_tax_id'] = df.groupby('merchant_tax_id')['merchant_market_hierarchy_id'].transform('nunique')
    df['qtd_mmhid_tax_id'] = df.groupby('tam_id')['qtd_mmhid_tax_id'].transform('max')


    df = assign_group_ids(df, ['merchant_tax_id', 'tam_id'], final_col='grouped_tax_id')

    df['qtd_nomes'] = df.groupby('group_id')['nome_master'].transform('nunique')
    return df


In [18]:
def main_data_treat_muni(df, file_name):

    """
    Main function to process data with options to display intermediate DataFrames and print verbose messages.

    Parameters:
    - read_data_func: Function to read initial data.
    - display_flag: Boolean flag indicating whether to display the DataFrame after each processing step.
    - verbose: Boolean flag indicating whether to print verbose messages during processing.
    """
    display_flag=False
    
    len_df = len(df)
    
    #if len_df> 100000:
    #    verbose = True
    #else:
    #    verbose = False
    verbose = True
    LEVEL_GROUP = ['cod_muni']
    
    
    print(f'Starting main data treat for file {file_name}, with size {len(df)}')

    start = time.time()
    execute_ctx = execute_with_context(LEVEL_GROUP, display_flag=display_flag, verbose=verbose)

    df = execute_ctx(init_group_id, df)
    df = execute_ctx(deal_unmerged_places, df)
    df = execute_ctx(deal_merged_places, df)
    df = execute_ctx(deal_merged_docs, df)
    df = execute_ctx(choose_prefered_document, df)
    df = execute_ctx(create_agrupamento_inspecao, df)
    df = execute_ctx(grouped_subs_asterisk, df)
    df = execute_ctx(final_ajustes, df)
    df = execute_ctx(create_doc_final, df)
    
    print(f'Main data treat Execution time file {file_name}: {time.time() - start:.2f} seconds')
    return df

In [19]:

def inter_data_treat_muni(df, file_idx):

    len_df = len(df)
    
    #if len_df> 100000:
    #    verbose = True
    #else:
    #    verbose = False
    verbose = True
    LEVEL_GROUP = ['cod_muni']
    
    cod_munis = list(df['cod_muni'].unique())
    
    start = time.time()
    if len(cod_munis) > 3:
        print(f'Starting inter data treat for qtd munis {len(cod_munis)}, with size {len(df)}')
    else:
        print(f'Starting inter data treat for munis {cod_munis}, with size {len(df)}')
        

    df = add_grouped_part(df, cod_munis) 
   
    print(f'Inter data treat Execution time file {file_idx}: {time.time() - start:.2f} seconds')
    return df

def save_intermediary(df, idx):
    df = inter_data_treat_muni(df, idx)
    df.to_parquet(f'data/intermediary/part_{idx}.parquet')
    print(f'File part_{idx}.parquet Saved')
                  

def save_tratados(file_name, input_date):
    df = pd.read_parquet(f'data/intermediary/{input_date}/{file_name}')
    df = main_data_treat_muni(df, file_name)
    df.to_parquet(f'data/tratados/{input_date}/{file_name}')
    print(f'File {file_name} Saved in Tratados')
              
def save_all_tratados(input_date):
    output_dir = f'data/intermediary/{input_date}'
    files = [x for x in os.listdir(output_dir) if x.endswith('parquet')]
    for f in files:
        save_tratados(f, input_date)

        
        
        
def accumulate_and_apply_v0():

    """
    Accumulates rows of a DataFrame based on a unique identifier and applies a function 
    to the accumulated DataFrame whenever a minimum row threshold is reached.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    func (function): The function to apply to the accumulated DataFrame.
    id_column (str): The name of the column with unique identifiers.
    min_rows_per_file (int): The minimum number of rows required before applying the function.
    """
    
    
    func = save_intermediary
    min_rows_per_file = 500000
    id_column = 'cod_muni'
    
    # Initialize the current file index and the DataFrame to accumulate rows
    current_file_index = 39
    accumulated_df = pd.DataFrame()
    len_accu = 0 
    
    
    #df = main_read_data()
    df = read_data('main_data')
    # Loop through each unique value in the identifier column
    
    print('Main Data Loaded')
    display(df)
    print(df.columns)
    
    
    def find_saved_munis():
        output_dir = r'data/intermediary'
        df = pd.concat([pd.read_parquet(f"{output_dir}/{f}", columns=['cod_muni']).drop_duplicates() for f in os.listdir(output_dir)])
        saved_munis = df['cod_muni'].tolist()
        return saved_munis
    
    df = df[~df['cod_muni'].isin(find_saved_munis())] 
    
    
    
    for unique_id in df[id_column].unique():
        # Subset the DataFrame for the current unique identifier
        subset_df = df[df[id_column] == unique_id]
        print(f'Filter {unique_id}')
        df = df[df[id_column] != unique_id]
        
        len_subset_df = len(subset_df)

        # Check if the accumulated DataFrame and the subset exceed the minimum row threshold
        if len_accu + len_subset_df > min_rows_per_file:
            # Apply the function to the accumulated DataFrame
            func(accumulated_df, current_file_index)
            # Increment the file index and reset the accumulated DataFrame
            current_file_index += 1
            accumulated_df = subset_df
            len_accu = len(accumulated_df)
        else:
            # Concatenate the subset DataFrame to the accumulated DataFrame
            accumulated_df = pd.concat([accumulated_df, subset_df], ignore_index=True)
            len_accu = len(accumulated_df)
            
            
    
    # Apply the function to any remaining rows in the accumulated DataFrame
    if not accumulated_df.empty:
        func(accumulated_df, current_file_index)
        
def accumulate_and_apply_v2(func):
    """concat files in 'nomes_agrupados', based in 'dict_files' and apply function"""

    dict_files = create_agg_nomes_agrupados()

    df = read_data('main_data')


    len_df = len(df)
    verbose = True
    LEVEL_GROUP = ['cod_muni']
    
    execute_ctx = execute_with_context(LEVEL_GROUP, display_flag=display_flag, verbose=verbose)

    
    start = time.time()
    if len(cod_munis) > 3:
        print(f'Starting inter data treat for qtd munis {len(cod_munis)}, with size {len(df)}')
    else:
        print(f'Starting inter data treat for munis {cod_munis}, with size {len(df)}')
    
    #df = add_grouped_part(df, cod_munis) 
    df = execute_ctx(add_grouped, df)

    print(f'Inter data treat Execution time file {file_idx}: {time.time() - start:.2f} seconds')

    list_cod_munis = [v for k, v in dict_files.items()]
    dfs = [df[df['cod_muni'].isin(cod_munis)] for cod_munis in list_cod_munis]

    for df in dfs:
        df.to_parquet('data/nomes_agrupados/df.parquet', index=False)

    


In [20]:
import os
import pandas as pd
import pyarrow.parquet as pq

def generate_parquet_summary(directory):
    # Lista todos os arquivos no diretório
    files = [f for f in os.listdir(directory) if f.endswith('.parquet')]
    
    # Cria listas para armazenar os nomes dos arquivos e a contagem de linhas
    file_names = []
    line_counts = []
    
    # Itera sobre cada arquivo, conta as linhas e armazena as informações
    for file in files:
        file_path = os.path.join(directory, file)
        parquet_file = pq.ParquetFile(file_path)
        file_names.append(file)
        line_counts.append(parquet_file.metadata.num_rows)
    
    # Cria o DataFrame
    summary_df = pd.DataFrame({
        'file_name': file_names,
        'line_count': line_counts
    })
    
    return summary_df



def calculate_group_ids(column):
    # Inicializa listas para soma cumulativa e IDs
    stopped_cumsum = []
    group_ids = []
    
    # Variáveis auxiliares
    cumsum = 0
    group_id = 1
    
    # Itera pelos valores da coluna
    for value in column:
        if cumsum + value > 1000000:
            group_id += 1
            cumsum = 0
        
        cumsum += value
        stopped_cumsum.append(cumsum)
        group_ids.append(group_id)
        
        if cumsum == 5:
            cumsum = 0
            group_id += 1
    
    return pd.Series(group_ids, index=column.index)


def create_agg_nomes_agrupados(input_date):
    # Exemplo de uso
    directory = f'data/nomes_agrupados/{input_date}'
    df = generate_parquet_summary(directory)
    df = df.sort_values('line_count')
    df['id'] = calculate_group_ids(df['line_count'])
    df['cod_muni'] = df['file_name'].apply(lambda filename: filename.split('_')[1].split('.')[0])

    import json
    dict_files = df.groupby('id')['cod_muni'].apply(lambda x: list(x)).to_dict()
    return dict_files

        

In [21]:

 
def accumulate_and_apply(input_date):
    
    def save_func(df, idx, cod_munis, input_date):
        print(f'Starting inter data treat for file {idx}')
        start = time.time()
        df = add_grouped_part(df, input_date, cod_munis)
        df.to_parquet(f'data/intermediary/{input_date}/part_{idx}.parquet')
        print(f'Inter data treat Execution time: {time.time() - start:.2f} seconds')
        print(f'File part_{idx}.parquet Saved') 
    
    """concat files in 'nomes_agrupados', based in 'dict_files' and apply function"""

    dict_files = create_agg_nomes_agrupados(input_date)
    dict_files = {k: [int(x) for x in v] for k, v in dict_files.items()}

    df = read_data(f'main_data_{input_date}')
    print(f'File Read: {input_date}')
    [save_func(df[df['cod_muni'].isin(v)], k, v, input_date) for k, v in dict_files.items()]
    


In [22]:
#accumulate_and_apply('2024-04-30')

In [24]:
#%load_ext line_profiler
#%lprun -f deal_merged_places deal_merged_places(df, LEVEL_GROUP)

In [25]:

import sys, importlib
importlib.reload(sys.modules['group_tam_id'])
from group_tam_id import (
    assign_group_ids,
    open_ton, load_to_gbq, create_doc_final, open_init_data,
    execute_with_context,
    init_group_id, 
    deal_merged_places, 
    deal_merged_docs,
    deal_unmerged_places, 
    choose_prefered_document, 
    create_agrupamento_inspecao, 
    grouped_subs_asterisk, 
    final_ajustes)

In [26]:
#save_all_tratados('2024-04-30')

In [28]:
print('Fim')

Fim


In [32]:
import pandas as pd
from google.cloud import bigquery

def upload_partitioned_table_to_bigquery(df, destination_table):
    """
    Uploads a DataFrame to BigQuery with partitioning on 'ingestion_date' column.
    If the table exists, it appends the data. If not, it creates the table.

    Args:
        df (pd.DataFrame): DataFrame to upload.
        destination_table (str): Destination table in the format 'project.dataset.table'.

    Returns:
        None
    """
    # Add 'ingestion_date' column
    df['ingestion_date'] = datetime.now()

    # Create BigQuery client
    client = bigquery.Client()

    # Define job configuration
    job_config = bigquery.LoadJobConfig(
        time_partitioning=bigquery.TimePartitioning(
            type_=bigquery.TimePartitioningType.DAY,
            field="ingestion_date"
        ),
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND  # Append data if table exists, otherwise create table
    )

    # Load DataFrame to BigQuery
    job = client.load_table_from_dataframe(df, destination_table, job_config=job_config)

    # Wait for the job to complete
    job.result()
    print("Data successfully loaded into BigQuery")
    
    
import os
import pandas as pd
import json

def load_available_data(directory, destination_table, final_data_treat_func, track_file='uploaded_files.json', restart_track_file=True):
    """
    Loads data from parquet files in the given directory and uploads them to a BigQuery table.
    Tracks uploaded files to avoid re-uploading.

    Args:
        directory (str): The directory containing parquet files.
        destination_table (str): The BigQuery table to upload data to.
        track_file (str): The path to the JSON file used to track uploaded files.

    Returns:
        None
    """
    # Load the list of uploaded files
    if os.path.exists(track_file) and not restart_track_file:
        with open(track_file, 'r') as f:
            uploaded_files = json.load(f)
    else:
        uploaded_files = []

    # Get list of files in the directory
    files = [f for f in os.listdir(directory) if f.endswith('parquet')]

    for file in files:
        if file not in uploaded_files:
            print(f'Opening {file}...')
            df = pd.read_parquet(os.path.join(directory, file))
            
            df = final_data_treat_func(df)
            
            print(f'Sending {file} to {destination_table}')
            upload_partitioned_table_to_bigquery(df, destination_table)
            
            print(f'{file} sent to {destination_table} successfully!')

            # Update the list of uploaded files and save it
            uploaded_files.append(file)
            with open(track_file, 'w') as f:
                json.dump(uploaded_files, f)
        else:
            print(f'{file} has already been uploaded. Skipping.')


def final_data_treat(df):
    if '_merge' in df.columns:
        df = df.drop(columns=['_merge'])             
    else:
        print('_merge not in df')
    
    if 'reference_month_x' in df.columns:
        df['reference_month'] = df['reference_month_x']
        del df['reference_month_x'], df['reference_month_y']
             
    #df['group_id_index'] = (df['group_id_index'].astype(str) + df['cod_muni'].astype(str)).astype(int)
    
    #df = df.drop(columns = ['inicio'])
             
    return df

                 


In [24]:
#load_available_data(directory='data/tratados/2024-04-30', final_data_treat_func=final_data_treat, destination_table='dataplatform-prd.master_contact.temp_nomes_agrupados')

In [25]:
print('Fim Upload')

Fim Upload


In [26]:

def open_all_intermediary(directory)
    """
    Loads data from parquet files in the given directory and uploads them to a BigQuery table.
    Tracks uploaded files to avoid re-uploading.

    Args:
        directory (str): The directory containing parquet files.
        destination_table (str): The BigQuery table to upload data to.
        track_file (str): The path to the JSON file used to track uploaded files.

    Returns:
        None
    """
    # Load the list of uploaded files
    if os.path.exists(track_file):
        with open(track_file, 'r') as f:
            uploaded_files = json.load(f)
    else:
        uploaded_files = []

    # Get list of files in the directory
    files = [f for f in os.listdir(directory) if f.endswith('parquet')]

    for file in files:
        if file not in uploaded_files:
            print(f'Opening {file}...')
            df = pd.read_parquet(os.path.join(directory, file))
            
            df = final_data_treat_func(df)
            
            print(f'Sending {file} to {destination_table}')
            upload_partitioned_table_to_bigquery(df, destination_table)
            
            print(f'{file} sent to {destination_table} successfully!')

            # Update the list of uploaded files and save it
            uploaded_files.append(file)
            with open(track_file, 'w') as f:
                json.dump(uploaded_files, f)
        else:
            print(f'{file} has already been uploaded. Skipping.')


SyntaxError: invalid syntax (1519832948.py, line 1)

In [27]:
def test(df):
    display_flag=False

    len_df = len(df)

    verbose = True
    LEVEL_GROUP = ['cod_muni']

    cod_munis = list(df['cod_muni'].unique())


    if len(cod_munis) > 3:
        print(f'Starting main data treat for qtd munis {len(cod_munis)}, with size {len(df)}')
    else:
        print(f'Starting main data treat for munis {cod_munis}, with size {len(df)}')

    start = time.time()
    execute_ctx = execute_with_context(LEVEL_GROUP, display_flag=display_flag, verbose=verbose)

    df = execute_ctx(init_group_id, df)
    df = execute_ctx(deal_unmerged_places, df)
    df = execute_ctx(deal_merged_places, df)
    
    df = execute_ctx(choose_prefered_document, df)
    df = execute_ctx(create_agrupamento_inspecao, df)
    df = execute_ctx(grouped_subs_asterisk, df)
    #df = execute_ctx(final_ajustes, df)
    print(f'Main data treat Execution time: {time.time() - start:.2f} seconds')
    return df

In [28]:

def execute(func, df, display_flag=True, verbose=True, LEVEL_GROUP=None, use_level=False, **kwargs):
    """
    Executes a given function on the DataFrame, optionally displays the result, and shows execution time.

    Parameters:
    - func: The processing function to execute. It should accept a DataFrame and return a DataFrame.
    - df: The DataFrame to process.
    - display_flag: Boolean flag indicating whether to display the DataFrame after processing.
    - verbose: Boolean flag indicating whether to print execution details.
    - LEVEL_GROUP: Optional parameter to pass a level group to the function.
    - use_level: Boolean flag indicating whether to use the LEVEL_GROUP parameter.
    - **kwargs: Additional keyword arguments to pass to the function.

    Returns:
    - The processed DataFrame.
    """

    start_time = time.time()  # Start timing before function execution
  

    if verbose:
        print(f'_______________\nExecuting function {func.__name__}...')

    if use_level:
        df = func(df, LEVEL_GROUP=LEVEL_GROUP, **kwargs)
    else:
        df = func(df, **kwargs)

    end_time = time.time()  # End timing after function execution
    execution_time = end_time - start_time  # Calculate execution time

    if verbose:
        print(f'Function {func.__name__} executed successfully in {execution_time:.2f} seconds.')
    
    
      
    if 'group_id' in df.columns:
        s = df['group_id'].str.len()
        print(f'Result Max group id size: {s.max()}')
        
        display(df.groupby('group_id').agg({
            #'group_id': 'nunique',
            'merchant_market_hierarchy_id': 'nunique'
        }).max())
        
        
    if 'group_id_index' in df.columns:
        sizes = df.groupby('group_id_index').ngroups
        print(f'qtd names group id: {sizes}')
        
        
    #if 'group_id' in df.columns: print(f'Number of groups: {df["group_id"].nunique()}')
    
    if display_flag:
        display(df)
    return df


def execute_with_context(LEVEL_GROUP, display_flag=False, verbose=False):
    import inspect
    def execute_context(func, df, **kwargs):
        # Check if 'LEVEL_GROUP' is in the function's parameters
        params = inspect.signature(func).parameters
        if 'LEVEL_GROUP' in params:
            # If 'LEVEL_GROUP' is a parameter, pass it along with 'df'
            result = execute(func, df, display_flag=display_flag, verbose=verbose, LEVEL_GROUP=LEVEL_GROUP,
                             use_level=True, **kwargs)
        else:
            # If 'LEVEL_GROUP' is not a parameter, call the function without it
            result = execute(func, df, display_flag=display_flag, verbose=verbose, use_level=False, **kwargs)
        return result
    return execute_context